In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
#import Segmentation
import time
import preds

importing Jupyter notebook from preds.ipynb
importing Jupyter notebook from Captcha_seg.ipynb
1080


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 864 samples, validate on 216 samples
Epoch 1/20
864/864 [==============================] - 26s 31ms/step - loss: 3.5891 - acc: 0.0208 - val_loss: 3.5855 - val_acc: 0.0185
Epoch 2/20
864/864 [==============================] - 14s 16ms/step - loss: 3.5484 - acc: 0.0544 - val_loss: 3.4385 - val_acc: 0.1204
Epoch 3/20
864/864 [==============================] - 14s 16ms/step - loss: 2.7651 - acc: 0.2373 - val_loss: 2.3053 - val_acc: 0.3380
Epoch 4/20
864/864 [==============================] - 13s 15ms/step - loss: 1.7703 - acc: 0.4815 - val_loss: 1.8952 - val_acc: 0.4815
Epoch 5/20
864/864 [==============================] - 13s 15ms/step - loss: 1.2015 - acc: 0.6343 - val_loss: 1.6375 - val_acc: 0.5231
Epoch 6/20
864/864 [==============================] - 12s 13ms/step - loss: 0.8468 - acc: 0.7454 - val_loss: 1.6958 - val_acc: 0.5880
Epoch 7/20
864/864 [==============================] - 11s 13ms/step - loss: 0.5516 - acc: 0.8345 - val_loss: 1.7981 - val_acc: 0.5926
Epoch 8/20
864/8

In [2]:
CATEGORIES = set()
names =[]
for i in os.listdir('train/'):
    names.append(i.split('.')[0])
for i in names:
    for x in list(i):
        CATEGORIES.add(x)
CATEGORIES=list(CATEGORIES)
print(CATEGORIES)

['Z', '7', '0', '5', 'F', 'L', 'b', 'p', '2', 's', 'P', 'r', 'E', 'k', 'Y', 'z', 'D', '6', 'd', 'H', 'S', 'K', 'f', '4', 'A', 'U', 'm', 'R', 'l', 'Q', 'V', 'j', 'h', 'i', 'w', 'J', 't', 'n', 'C', 'v', 'X', 'y', 'c', 'W', 'e', 'I', 'O', 'g', 'a', 'N', '1', 'q', 'T', 'o', 'x', 'M', 'B', '3', 'G', 'u', '9', '8']


In [3]:
#print(len(train_data))

In [4]:
import random

#random.shuffle(train_data)

In [5]:
#for sample in train_data[:10]:
#    print(sample[1])

In [6]:
X = []
y = []

for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(19))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


In [7]:
#loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

In [8]:
#print(loss_value, metrics_value)

In [9]:
import random

In [10]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
done = []
list_ = preds.predictions('test',CATEGORIES)
CAPTCHA_DIR = list_[0]
for x in CAPTCHA_DIR:
    captchas = random.choice(CAPTCHA_DIR)
    if captchas in done:
        continue
    else:
        done.append(captchas)
        start_time = time.time()
        predict_list= []
        print('\n')
        print('Actual: ',captchas[0][0])

        files = []
        for segments in list_:
            files.append(segments)

        #for segments in files:
        #    filepath = (SEGMENTS_DIR,segments)
        #    prediction = model.predict([prepare(filepath)])
        #    predict_list.append(CATEGORIES[np.argmax(prediction[0])])

        elapsed_time = time.time()-start_time
        time.sleep(captchas[1])
        total_time += elapsed_time
        final_prediction = ''.join(predict_list)
        #if captcha_name[0]== final_prediction:
        #    score +=1
        score = list_[1]
        print('Predicted:',captchas[0][1],'\n')
        print('Elapsed time= ',captchas[1],'s')
        total_time +=captchas[1]
        print('<------------------------------->')    
    

print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',len(list_[0]))
print('Total time taken= ',total_time,'s')
print('Accuracy= ', (score/len(list_[0])*100))
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  XJWS
Predicted: XJWS 

Elapsed time=  0.19199604670353543 s
<------------------------------->


Actual:  KQGHPS
Predicted: KQGHPS 

Elapsed time=  0.5728025642643294 s
<------------------------------->


Actual:  WS4DW
Predicted: MyQR 

Elapsed time=  0.4707071170945978 s
<------------------------------->


Actual:  3XSW
Predicted: 3XSW 

Elapsed time=  0.6446003874158373 s
<------------------------------->


Actual:  QQS3
Predicted: TxyP 

Elapsed time=  0.8173675065600204 s
<------------------------------->


Actual:  HVHRWR
Predicted: HVHRWR 

Elapsed time=  0.6406374436936368 s
<------------------------------->


Actual:  dH3dH4
Predicted: dH3dH4 

Elapsed time=  0.1625602176254386 s
<------------------------------->


Actual:  MXVSP
Predicted: nnPx 

Elapsed time=  0.5913858289187283 s
<------------------------------->


Actual:  KWS53J
Predicted: vf8u 

Elapsed time=  0.5168992060095468 s
<------------------------------->


Actual:  KY5VWd
Predicted: KY5VWd 

Elapsed t

Predicted: HdVD3 

Elapsed time=  0.4330092077872407 s
<------------------------------->


Actual:  QR5W
Predicted: QR5W 

Elapsed time=  0.07335354954148143 s
<------------------------------->


Actual:  ND554
Predicted: yyXs 

Elapsed time=  0.9229458005712903 s
<------------------------------->


Actual:  QMRDd
Predicted: QMRDd 

Elapsed time=  0.6394413661225381 s
<------------------------------->


Actual:  VKWPGX
Predicted: OdaK 

Elapsed time=  0.7618033453517963 s
<------------------------------->


Actual:  VYPHG4
Predicted: 46qO 

Elapsed time=  0.7875485736330549 s
<------------------------------->


Actual:  HX33M6
Predicted: lc3A 

Elapsed time=  0.26561706535192786 s
<------------------------------->


Actual:  PKKSP
Predicted: lQcw 

Elapsed time=  0.3913394497147691 s
<------------------------------->


Actual:  GWSW4
Predicted: wjfg 

Elapsed time=  0.8025627732000074 s
<------------------------------->


Actual:  VKWdV6
Predicted: Z0NG 

Elapsed time=  0.2468153478403

Predicted: rPRv 

Elapsed time=  0.5691610751706171 s
<------------------------------->


Actual:  GVPKVS
Predicted: GVPKVS 

Elapsed time=  0.5042195785555131 s
<------------------------------->


Actual:  PD4MH
Predicted: 3Udi 

Elapsed time=  0.09585480599749319 s
<------------------------------->


Actual:  6N3Y
Predicted: obcb 

Elapsed time=  0.05156756280815855 s
<------------------------------->


Actual:  K4PSd4
Predicted: 7YSP 

Elapsed time=  0.29518692667525603 s
<------------------------------->


Actual:  KJ6VJ
Predicted: KJ6VJ 

Elapsed time=  0.14140791605322167 s
<------------------------------->


Actual:  VWWMJ
Predicted: FjRF 

Elapsed time=  0.47699729556089865 s
<------------------------------->


Actual:  LVW6
Predicted: JBdW 

Elapsed time=  0.26229279085102475 s
<------------------------------->


Actual:  XdQKWK
Predicted: XdQKWK 

Elapsed time=  0.6195365042351925 s
<------------------------------->


Actual:  3G6GMd
Predicted: 3G6GMd 

Elapsed time=  0.83706

Predicted: 4DGP 

Elapsed time=  0.2177119888385387 s
<------------------------------->


Actual:  QJLSd4
Predicted: Xk9X 

Elapsed time=  0.5092522893273151 s
<------------------------------->


Actual:  QXLYRK
Predicted: QXLYRK 

Elapsed time=  0.29339081220056984 s
<------------------------------->


Actual:  WHPV
Predicted: ev3I 

Elapsed time=  0.8422561654608502 s
<------------------------------->


Actual:  XSXJ
Predicted: kTbZ 

Elapsed time=  0.1994187000179135 s
<------------------------------->


Actual:  4W4ML
Predicted: 4W4ML 

Elapsed time=  0.6508012800334818 s
<------------------------------->


Actual:  SX6Yd
Predicted: qTfF 

Elapsed time=  0.6561063938577743 s
<------------------------------->


Actual:  P5JHK
Predicted: P5JHK 

Elapsed time=  0.48484829826824205 s
<------------------------------->


Actual:  5KDP
Predicted: 5KDP 

Elapsed time=  0.3569306854263954 s
<------------------------------->


Actual:  XGSQ
Predicted: XGSQ 

Elapsed time=  0.0644178735301166

Predicted: sYrE 

Elapsed time=  0.8520041655771114 s
<------------------------------->


Actual:  WRPS
Predicted: WRPS 

Elapsed time=  0.8917102084023549 s
<------------------------------->


Actual:  5GKY
Predicted: EYIx 

Elapsed time=  0.8309469079643622 s
<------------------------------->
Total number of CAPTCHAS broken:  284
Total number of CAPTCHAS attempted:  493
Total time taken=  158.47544881964416 s
Accuracy=  57.606490872210955
